The evolution of a trajectory $ \mathbf{x}(t;t0,\mathbf{x_0}) $ as a function of its initial position $ \mathbf{x_0} $ is described by the flow map

$$ \mathbf{F}_{t_0}^t : \mathbf{x}_0 \rightarrow \mathbf{x}(t;t_0,\mathbf{x_0}) $$

Differentiation of the flow map $ {\mathbf{F}_1}_{t_0}^t = \begin{pmatrix} {F_1}_{t_0}^t \\ {F_2}_{t_0}^t \end{pmatrix} $ with respect to time gives:

\begin{equation}
\dfrac{d}{dt}\mathbf{F}_{t_0}^t(\mathbf{x_0}) = \mathbf{v}(\mathbf{F}_{t_0}^t(\mathbf{x_0}), t) = \begin{pmatrix} u((\mathbf{F}_{t_0}^t(\mathbf{x_0}), t)) \\ v((\mathbf{F}_{t_0}^t(\mathbf{x_0}), t)) \end{pmatrix}
\label{eq: dFdt},
\end{equation}

with $ \mathbf{v}(\mathbf{F}_{t_0}^t(\mathbf{x_0}), t) $ being the velocity field.

Taking the gradient of \ref{eq: dFdt}, it holds that $ D \mathbf{F}_{t_0}^t(\mathbf{x_0}) $ evolves according to the equations of variations:

\begin{equation}
\dfrac{d}{dt} D \mathbf{F}_{t_0}^t(\mathbf{x_0}) = D\mathbf{v}(\mathbf{F}_{t_0}^t(\mathbf{x_0}), t) D \mathbf{F}_{t_0}^t(\mathbf{x_0}) = \begin{pmatrix} \partial_x u && \partial_y u \\ \partial_x v && \partial_y v \end{pmatrix} \begin{pmatrix} D_x{F_1}_{t_0}^t && D_y{F_1}_{t_0}^t \\ D_x{F_2}_{t_0}^t && D_y{F_2}_{t_0}^t \end{pmatrix}
\label{eq: dDFdt}
\end{equation}

Equations \ref{eq: dFdt} and \ref{eq: dDFdt} lead to a system of coupled ODEs.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| t | float | time |
| x | array (2,) | $ \mathbf{x} $ |
| X | array (Ny, Nx) | X-meshgrid|
| Y | array (Ny, Nx) | Y-meshgrid|
| Interpolant | list (2,) | Interpolant[0]: Interpolant object for $ u((\mathbf{x}, t))  $ <br /> Interpolant[1]: Interpolant object for $ v((\mathbf{x}, t))  $. <br /> <br /> The interpolation, however, works only <br /> if the particle is in the defined flow domain. <br /> At the boundary of the domain, <br /> the interpolant is not defined.|
| periodic | list (2,) | periodic[0]: periodicity in x <br /> periodic[1]: periodicity in y <br />|
| defined_domain | array (Nx, Ny) | points on the meshgrid where velocity field is defined |
| bool_unsteady | bool | specifies if velocity field is unsteady/steady |
| dt_data | float | time spacing of the velocity data |
| delta | list (2,) | delta[0]: x-grid spacing of the velocity data <br /> delta[1]: y-grid spacing of the velocity data|
| u | float | $ u(\mathbf{x}, t) $ |
| v | float | $ v(\mathbf{x}, t) $ |
| aux_grid | list (2,) | aux_grid[0]: dx_auxiliary <br /> aux_grid[1]: dy_auxiliary |
| DxF1 | float | $ D_x{F_1}_{t_0}^t(\mathbf{x_0}) $ |
| DyF1 | float | $ D_y{F_1}_{t_0}^t(\mathbf{x_0}) $ |
| DxF2 | float | $ D_x{F_2}_{t_0}^t(\mathbf{x_0}) $ |
| DyF2 | float | $ D_y{F_2}_{t_0}^t(\mathbf{x_0}) $ |

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-1])

# add Algorithm folder to current working path in order to access the functions
sys.path.append(parent_directory+"/utils")

In [2]:
# Import numpy
import numpy as np

# Import function that computes velocity
from ipynb.fs.defs.velocity import velocity

# Import function that computes gradient of velocity
from ipynb.fs.defs.gradient_velocity import gradient_velocity

In [3]:
def dFdt_eqVar(t, x, X, Y, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta, aux_grid):
    
    # Flow map F at time t
    F = x[:2]
    
    # DF11
    DxF1 = x[2]
    
    # DF12
    DyF1 = x[3]
    
    # DF21
    DxF2 = x[4]
    
    # DF22
    DyF2 = x[5]
    
    # DF
    DF = np.array([[DxF1, DyF1], [DxF2, DyF2]])
    
    dx_data = delta[0]
    dy_data = delta[1]
    
    # check if periodic in x
    if periodic[0]:
        
        x[0] = x[0]%(X[0, -1, 0]-X[0, 0, 0])
    
    # check if periodic in y
    if periodic[1]:
        
        x[1] = x[1]%(Y[-1, 0, 0]-Y[0, 0, 0])
    
    # compute velocity
    vel = velocity(t, F, X, Y, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta)
    
    # compute gradient of velocity
    grad_vel = gradient_velocity(t, F, X, Y, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta, aux_grid)
    
    # eq. 2
    dDFdt = grad_vel@DF
    
    # eq.1
    u = vel[0]
    v = vel[1]
    
    # extract elements of dDFdt
    dDxF1dt = dDFdt[0,0]
    dDyF1dt = dDFdt[0,1]
    dDxF2dt = dDFdt[1,0]
    dDyF2dt = dDFdt[1,1]
    
    return [u, v, dDxF1dt, dDyF1dt, dDxF2dt, dDyF2dt]
    